In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minRR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxRR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["RR"].mean()
    or_std  = d["RR"].std(ddof=1)
    df_out.loc[idx, "RR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minRR"] = d["2.5%"].min()
    df_out.loc[idx, "maxRR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
data = pd.read_parquet('data/BBAG-long.parquet')

In [4]:
Counter(data.country)

Counter({'Austria': 4320,
         'Belgium': 4942,
         'Czechia': 5575,
         'Switzerland': 2634,
         'Germany': 4567,
         'Denmark': 2609,
         'Estonia': 7522,
         'Spain': 4912,
         'France': 3640,
         'Hungary': 3010,
         'Italy': 4541,
         'Netherlands': 2546,
         'Poland': 3612,
         'Portugal': 1914,
         'Sweden': 2792,
         'Slovenia': 5291,
         'Luxembourg': 2104,
         'Greece': 2599,
         'Croatia': 2798,
         'Bulgaria': 1953,
         'Cyprus': 1205,
         'Finland': 1982,
         'Lithuania': 2033,
         'Latvia': 1684,
         'Malta': 1259,
         'Romania': 2083,
         'Slovakia': 2022})

In [5]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


In [6]:
data.columns

Index(['record_id', 'country', 'Sex_1F_2M', 'Education', 'Barthel',
       'Diabetes_1Y_0N', 'Hypertension_1Y_0N', 'Heart_Disease_1Y_0N',
       'Physical_activity_1Y_0N', 'Cognition', 'Well_being_domain',
       'Sleep_problems_1Y_0N', 'Audition_problems', 'Vision_problems', 'Age',
       'Mono', 'One', 'Two', 'Three', 'Total', 'total_exposomes',
       'sociopolitical', 'social_physical', 'GDP', 'GDP_per_capita',
       'gender_equal_l', 'Polution_conc_inv', 'Eq', 'free_parties_l',
       'inclu_suff_est', 'cred_elect_est', 'local_dem_est', 'Migration',
       'number_leng_inst', 'number_stable_inst', 'distance', 'Proficiency',
       'Educ_doct', 'Educ_upper_sec', 'Educ_master', 'Educ_bach',
       'year_bilingues', 'delta_time', 'GAP_corrected', 'GAP_bin',
       'y_pred_corrected'],
      dtype='object')

## Odd ratios with macrosocials (new vars)

### Without Iteration

In [7]:
results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP'
    
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:


        c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
        c_results_merge_df_all.reset_index(drop = True, inplace = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [8]:
for covar in covar_list:
    c_dfRR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_new-covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfRR)

GDP_per_capita


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
0,1.217742,1.307343,1.261747,12.836560,1.023213e-37,Mono,GDP_per_capita
1,0.835581,0.918387,0.876006,-5.491802,3.978537e-08,One,GDP_per_capita
2,0.781228,0.837831,0.809035,-11.875469,1.587413e-32,Two,GDP_per_capita
3,0.819299,0.918154,0.867319,-4.898322,9.665871e-07,Three,GDP_per_capita
4,0.764308,0.820651,0.791978,-12.853218,8.250566e-38,Total,GDP_per_capita


GDP


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
5,1.434527,1.534246,1.483549,23.007191,3.949366e-117,Mono,GDP
6,0.891888,0.979211,0.934530,-2.841613,4.488590e-03,One,GDP
7,0.718794,0.774481,0.746119,-15.385472,2.048761e-53,Two,GDP
8,0.597610,0.664725,0.630275,-17.000421,8.153193e-65,Three,GDP
9,0.651147,0.696477,0.673431,-23.028697,2.405126e-117,Total,GDP


### 1000-Iteration

In [9]:

results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP' 
    
)


n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
            c_results_merge_df_all.reset_index(drop = True, inplace = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["RR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)



In [10]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [11]:
for covar in covar_list:
    c_dfRR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_new-covar-' + covar + '-1000-iter.xlsx')

    print(covar)
    display(c_dfRR)

GDP_per_capita


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
0,GDP_per_capita,Mono,0.286518,0.613504,0.996295,1.252563,1.251511,1.253615,1.211075,1.294732,10.0
1,GDP_per_capita,One,0.142282,0.544040,0.995612,0.871323,0.869966,0.872680,0.832842,0.910840,10.0
2,GDP_per_capita,Two,0.273007,0.401946,0.995194,0.817632,0.817062,0.818202,0.791823,0.844812,10.0
3,GDP_per_capita,Three,0.333219,0.612461,0.997541,0.875952,0.874575,0.877328,0.830285,0.923974,10.0
4,GDP_per_capita,Total,0.285414,0.613139,0.996323,0.797802,0.797132,0.798473,0.771778,0.825183,10.0


GDP


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
5,GDP,Mono,0.123297,0.436031,0.995325,1.473368,1.471836,1.474901,1.427658,1.521877,10.0
6,GDP,One,0.220829,0.491304,0.987590,0.926443,0.925131,0.927755,0.886398,0.968498,10.0
7,GDP,Two,0.176444,0.235910,0.985425,0.756500,0.755985,0.757015,0.731041,0.783079,10.0
8,GDP,Three,0.176630,0.495974,0.992617,0.637840,0.636651,0.639030,0.605887,0.670647,10.0
9,GDP,Total,0.117313,0.429470,0.994621,0.678091,0.677387,0.678796,0.656450,0.699833,10.0


## Odd ratios with macrosocials (new vars-several covars)

### Without Iteration

In [12]:
results_merge_df_all = data.copy()

covar_name_group = ['globals',
                    'phisicals',
                    'sociopoliticals',
                    'economics',
                    'linguistics',
                    ]

covar_list = (['sociopolitical', 'social_physical'], 
              ['gender_equal_l', 'Polution_conc_inv', 'Eq'],
              ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est'],
              ['GDP', 'GDP_per_capita'],
              ['number_leng_inst', 'number_stable_inst', 'distance', 'Migration'],
              
)

df_directions_odd_all = []

for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]

    #print(covar, covar_name)
    
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all.dropna(subset= [i] + covar + ['delta_time', 'GAP_bin'])
        c_results_merge_df_all.reset_index(drop = True, inplace = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i] + covar + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar_name
        df_directions_odd_all.append(df_i)


df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [13]:
for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]
    
    c_dfRR = df_directions_odd[df_directions_odd.Covar == covar_name]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_new-covar-several-covars-' + covar_name + '.xlsx')

    print(covar)
    display(c_dfRR)

['sociopolitical', 'social_physical']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
0,1.302609,1.414539,1.357421,14.531206,7.685538e-48,Mono,globals
1,0.860480,0.940737,0.899714,-4.645504,3.392464e-06,One,globals
2,0.853299,0.934625,0.893037,-4.871168,1.109407e-06,Two,globals
3,0.720614,0.802705,0.760553,-9.945090,2.649337e-23,Three,globals
4,0.706231,0.766932,0.735956,-14.574879,4.058402e-48,Total,globals


['gender_equal_l', 'Polution_conc_inv', 'Eq']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
5,1.366415,1.474893,1.419618,17.978905,2.850678e-72,Mono,phisicals
6,0.911068,0.999337,0.954182,-1.988073,4.680364e-02,One,phisicals
7,0.796240,0.863692,0.829281,-9.024059,1.812469e-19,Two,phisicals
8,0.635752,0.710478,0.672077,-14.017180,1.223825e-44,Three,phisicals
9,0.677399,0.731224,0.703797,-18.008649,1.666465e-72,Total,phisicals


['free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
10,1.320848,1.414084,1.366671,17.952412,4.595092e-72,Mono,sociopoliticals
11,0.917823,1.006016,0.960908,-1.703690,8.843897e-02,One,sociopoliticals
12,0.774769,0.832519,0.803125,-11.954403,6.157436e-33,Two,sociopoliticals
13,0.664721,0.734478,0.698730,-14.081825,4.912464e-45,Three,sociopoliticals
14,0.706520,0.756461,0.731064,-17.978627,2.865017e-72,Total,sociopoliticals


['GDP', 'GDP_per_capita']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
15,1.188894,1.282039,1.234588,10.951799,6.514072e-28,Mono,economics
16,0.767746,0.850558,0.808092,-8.154089,3.518211e-16,One,economics
17,0.797956,0.859213,0.828018,-10.001746,1.497327e-23,Two,economics
18,0.901949,1.032733,0.965128,-1.027534,3.041693e-01,Three,economics
19,0.779400,0.840564,0.809404,-10.971732,5.226177e-28,Total,economics


['number_leng_inst', 'number_stable_inst', 'distance', 'Migration']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
20,1.474155,1.613580,1.542293,18.793662,8.509933e-79,Mono,linguistics
21,0.915981,1.008345,0.961054,-1.620876,1.050442e-01,One,linguistics
22,0.715636,0.779275,0.746778,-13.435173,3.761882e-41,Two,linguistics
23,0.617952,0.698907,0.657184,-13.366797,9.452458e-41,Three,linguistics
24,0.618907,0.677568,0.647574,-18.809487,6.314645e-79,Total,linguistics


### 1000-Iteration

In [14]:
results_merge_df_all = data.copy()

covar_name_group = ['globals',
                    'phisicals',
                    'sociopoliticals',
                    'economics',
                    'linguistics',
                    ]

covar_list = (['sociopolitical', 'social_physical'], 
              ['gender_equal_l', 'Polution_conc_inv', 'Eq'],
              ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est'],
              ['GDP', 'GDP_per_capita'],
              ['number_leng_inst', 'number_stable_inst', 'distance', 'Migration'],
              
)

n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]

df_directions_odd_all = []

for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all.dropna(subset= [i] + covar + ['delta_time', 'GAP_bin'])
            c_results_merge_df_all.reset_index(drop = True, inplace = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i] + covar + ['delta_time']]


            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar_name, i, "y")
            key_p = (covar_name, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["RR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar_name
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)


df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)



In [15]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [16]:
for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]
    
    c_dfRR = df_directions_odd_f[df_directions_odd_f.Covar == covar_name]
    c_dfRR.to_excel('Results/long/long_RR_-with-slovakia_new-covar-several-covars-' + covar_name + '-1000-iter.xlsx')

    print(covar)
    display(c_dfRR)

['sociopolitical', 'social_physical']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
0,globals,Mono,0.194017,0.419228,0.921518,1.348789,1.347159,1.350420,1.298102,1.402516,10.0
1,globals,One,0.328069,0.292759,0.905449,0.894532,0.893326,0.895738,0.857654,0.932484,10.0
2,globals,Two,0.239970,0.176238,0.882511,0.906863,0.905828,0.907898,0.869153,0.946482,10.0
3,globals,Three,0.211843,0.310863,0.916626,0.765333,0.763964,0.766702,0.726899,0.805205,10.0
4,globals,Total,0.225349,0.493164,0.936731,0.740667,0.739773,0.741562,0.712292,0.769592,10.0


['gender_equal_l', 'Polution_conc_inv', 'Eq']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
5,phisicals,Mono,0.360748,0.368119,0.947140,1.432708,1.430199,1.435218,1.381402,1.486401,10.0
6,phisicals,One,0.278002,0.435999,0.959793,0.964960,0.963294,0.966627,0.923110,1.008453,10.0
7,phisicals,Two,0.191424,0.386544,0.959697,0.823463,0.822012,0.824914,0.791782,0.857339,10.0
8,phisicals,Three,0.243630,0.402513,0.916736,0.656763,0.655111,0.658414,0.622720,0.694068,10.0
9,phisicals,Total,0.365926,0.369724,0.956800,0.697372,0.696149,0.698595,0.672158,0.723289,10.0


['free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
10,sociopoliticals,Mono,0.352315,0.450493,0.994394,1.364850,1.363228,1.366473,1.321359,1.410636,10.0
11,sociopoliticals,One,0.124353,0.372627,0.988284,0.952949,0.951752,0.954146,0.912506,0.994605,10.0
12,sociopoliticals,Two,0.252735,0.325132,0.991120,0.810211,0.809544,0.810879,0.783779,0.837927,10.0
13,sociopoliticals,Three,0.161860,0.417179,0.992679,0.701931,0.700690,0.703172,0.668805,0.735803,10.0
14,sociopoliticals,Total,0.367144,0.456283,0.994179,0.732040,0.731170,0.732910,0.708252,0.756165,10.0


['GDP', 'GDP_per_capita']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
15,economics,Mono,0.236272,0.596401,0.997069,1.221412,1.220384,1.222441,1.178606,1.264299,10.0
16,economics,One,0.175064,0.647205,0.997391,0.800397,0.798871,0.801923,0.762223,0.840794,10.0
17,economics,Two,0.230238,0.576780,0.996760,0.839786,0.839247,0.840326,0.811604,0.869217,10.0
18,economics,Three,0.279219,0.631470,0.995961,0.986326,0.984415,0.988238,0.925139,1.050213,10.0
19,economics,Total,0.248672,0.592796,0.997340,0.818151,0.817461,0.818841,0.790348,0.847914,10.0


['number_leng_inst', 'number_stable_inst', 'distance', 'Migration']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
20,linguistics,Mono,0.276844,0.470159,0.995451,1.529270,1.527025,1.531515,1.465823,1.597197,10.0
21,linguistics,One,0.258590,0.415646,0.990627,0.953679,0.952311,0.955047,0.911456,0.998102,10.0
22,linguistics,Two,0.264031,0.354869,0.989329,0.757736,0.756986,0.758486,0.728122,0.788151,10.0
23,linguistics,Three,0.179047,0.437540,0.993658,0.664583,0.663183,0.665983,0.626674,0.704453,10.0
24,linguistics,Total,0.282496,0.480233,0.995464,0.653106,0.652148,0.654065,0.625271,0.681425,10.0
